# Some preliminary checks

In [1]:
import torch
import os

os.environ["RAY_DEDUP_LOGS"] = "0"

import ray 
print("Ray version :", ray.__version__)

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

print("MPS Available:", torch.backends.mps.is_available())

torch._dynamo.list_backends()

Ray version : 2.35.0
PyTorch Version: 2.4.0
CUDA Available: False
MPS Available: True


['cudagraphs', 'inductor', 'onnxrt', 'openxla', 'tvm']

### Important : Number of CPUs and GPUs available

In [2]:
import psutil

# print number of gpus / CPUs
print("Number of CPUs: ", psutil.cpu_count())

num_cpus = 12
num_gpus = 0
num_learner = 1

assert num_cpus <= psutil.cpu_count()

Number of CPUs:  12


# Environement and algorithm configuration

Some of the commented lines are preparation work to use a futur feature of RLLib

Note: In multi-agent environments, `rollout_fragment_lenght` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.


In [3]:
from ray.rllib.policy.policy import PolicySpec
#from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
#from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec

from ray.tune.registry import get_trainable_cls

from particle_2d_env import Particle2dEnvironment
from particle_2d_env import MetricsCallbacks, RenderingCallbacks
from config import run_config

ALGO = "PPO"        
FRAMEWORK= "torch" # "tf2" or "torch"
env = Particle2dEnvironment(run_config["env"])

config = (
    get_trainable_cls(ALGO).get_default_config()
    .environment(Particle2dEnvironment, env_config=run_config["env"])
    .framework(FRAMEWORK,)
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=False,
    )
    .callbacks(MetricsCallbacks)
    .callbacks(RenderingCallbacks)
    # Specify the learner's hyperparameters.
    .training(
        num_sgd_iter=5,          
        sgd_minibatch_size=512,             # the batch size (only for PPO)
        #minibatch_size = 256,                # the batch size (only for IMPALA)
        train_batch_size=524288,             # the number of step collected
        model={
            "fcnet_hiddens": [128, 128, 128], 
            "use_attention": True,
            #"use_lstm": False,
            #"max_seq_len": 5,
            #"lstm_cell_size": 16,
        },
        #lr=tune.grid_search([0.01, 0.001, 0.0001])
    )
    .multi_agent(
        policies= {
            "prey": PolicySpec(
                policy_class=None,  # infer automatically from Algorithm
                observation_space=env.observation_space[0],  # if None infer automatically from env
                action_space=env.action_space[0],  # if None infer automatically from env
                config={"gamma": 0.85},  # use main config plus <- this override here
            ),
            "predator": PolicySpec(
                policy_class=None,
                observation_space=env.observation_space[0],
                action_space=env.action_space[0],
                config={"gamma": 0.85},
            ),
        },
        policy_mapping_fn = lambda id, *arg, **karg: "prey" if env.agents[id].agent_type == 0 else "predator",
        policies_to_train=["prey", "predator"],
        count_steps_by="agent_steps",
    )
    .env_runners(
        rollout_fragment_length="auto", #"auto" for PPO explained here : https://docs.ray.io/en/latest/rllib/rllib-sample-collection.html
        batch_mode= 'truncate_episodes',
        num_env_runners=num_cpus-num_learner*num_gpus-1, # need 2 for IMPALA, 1 for PPO
        num_envs_per_env_runner=1,
    )
    .checkpointing(export_native_model_files=True)
)


/Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


# Training

In [4]:
from ray.rllib.policy.policy import Policy
from ray.rllib.algorithms.callbacks import DefaultCallbacks

path_to_checkpoint = None #os.getcwd() + "/ray_results" + "PPO_2024-05-18_00-08-19/PPO_Particle2dEnvironment_bb60c_00000_0_2024-05-18_00-08-19/checkpoint_000001"

def restore_weights(path, policy_type):
    checkpoint_path = os.path.join(path, f"policies/{policy_type}")
    restored_policy = Policy.from_checkpoint(checkpoint_path)
    return restored_policy.get_weights()

if path_to_checkpoint is not None: 
    class RestoreWeightsCallback(DefaultCallbacks):
        def on_algorithm_init(self, *, algorithm: "Algorithm", **kwargs) -> None:
            algorithm.set_weights({"predator": restore_weights(path_to_checkpoint, "predator")})
            algorithm.set_weights({"prey": restore_weights(path_to_checkpoint, "prey")})

    config.callbacks(RestoreWeightsCallback)



## Launch training

In [ ]:
from ray import train, tune
from ray.tune import Tuner
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.rllib.utils.test_utils import check_learning_achieved
import os

ray.init(
    num_cpus=num_cpus, 
    num_gpus=num_gpus
)

# Stop criterium
stop = {  
    "training_iteration": 1500,
    #"timesteps_total": 200000000,
}

# Read the API key from the file to use Wanddb
with open('wandb_api_key.txt', 'r') as file:
    api_key = file.read().strip()
callbacks = [
    WandbLoggerCallback(                   
        project="marl-rllib", 
        group=ALGO,
        api_key=api_key,
        log_config=True,
        upload_checkpoints=True
    ), 
]

# When to save the models 
checkpoint_config = train.CheckpointConfig(         
    checkpoint_at_end=True,
    checkpoint_frequency=10,
)

# Where to save 
# absolute path + ray_results directory
storage_path=os.getcwd() + "/ray_results"

if path_to_checkpoint is None : 
    tuner = tune.Tuner(
        ALGO,                                                 # Defined before
        param_space=config,                                   # Defined before
        run_config=train.RunConfig(    
            storage_path=storage_path,
            stop=stop,
            verbose=3,
            callbacks=callbacks,
            checkpoint_config=checkpoint_config,
        ),
    )
    # Run the experiment 
    results = tuner.fit()

# If we instantiate previously trained neural network
else: 
    callbacks.append(RestoreWeightsCallback)

    results = tune.run(
        ALGO,
        config=config.to_dict(),
        storage_path=storage_path,
        stop=stop,
        verbose=3,
        callbacks=callbacks,
        checkpoint_config=checkpoint_config,
    )


ray.shutdown()


2024-08-31 16:51:24,686	INFO worker.py:1783 -- Started a local Ray instance.
2024-08-31 16:51:25,276	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(RolloutWorker pid=19316) /Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=19316)   gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(RolloutWorker pid=19318) /Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=19318)   gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(RolloutWorker pid=19317) /Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=19317)   gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
(RolloutWorker pid=19320) /Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packag

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_media,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_sampled_this_iter,num_agent_steps_sampled_throughput_per_sec,num_agent_steps_trained,num_agent_steps_trained_this_iter,num_agent_steps_trained_throughput_per_sec,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_trained,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_Particle2dEnvironment_7ee3a_00000,4195691,"{'num_env_steps_sampled': 135066, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 4195691, 'num_agent_steps_trained': 0}",{},2024-08-31_17-07-03,False,{},MacBook-Pro-de-Tanguy.local,"{'learner': {'predator': {'vf_loss': 0.006170559208840132, 'num_trainable_parameters': 157445.0, 'num_module_steps_trained': 33964, 'curr_entropy_coeff': 0.0, 'mean_kl_loss': 0.016131693497300148, 'entropy': 2.8569726943969727, 'vf_loss_unclipped': 0.006170559208840132, 'curr_kl_coeff': 0.30000001192092896, 'policy_loss': -0.1064554750919342, 'total_loss': -0.0954454094171524, 'default_optimizer_learning_rate': 5e-05, 'vf_explained_var': 0.12016910314559937, 'num_non_trainable_parameters': 0.0}, 'prey': {'num_module_steps_trained': 490492, 'curr_entropy_coeff': 0.0, 'mean_kl_loss': 0.007941948249936104, 'entropy': 2.742658853530884, 'vf_loss_unclipped': 1.1772305697377305e-05, 'curr_kl_coeff': 0.05000000074505806, 'policy_loss': -0.08082485944032669, 'total_loss': -0.1758614033460617, 'vf_explained_var': 0.21066975593566895, 'default_optimizer_learning_rate': 5e-05, 'num_non_trainable_parameters': 0.0, 'vf_loss': 1.1772305697377305e-05, 'num_trainable_parameters': 157445.0}, '__all_modules__': {'total_loss': -0.1758614033460617, 'num_non_trainable_parameters': 0.0, 'num_trainable_parameters': 314890.0, 'num_env_steps_trained': 16982, 'num_module_steps_trained': 524456}}, 'num_env_steps_sampled': 135066, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 4195691, 'num_agent_steps_trained': 0}",8,127.0.0.1,4195691,4195691,524456,5544.99,0,0,0,135066,4195691,0,11,0,0,0,"{'cpu_util_percent': 51.826666666666675, 'ram_util_percent': 49.66484848484849}",19300,822.543,100.458,822.543,"{'training_iteration_time_ms': 97619.325, 'restore_workers_time_ms': 0.02, 'training_step_time_ms': 97619.264, 'sample_time_ms': 42556.841, 'synch_weights_time_ms': 19.942}",1725116823,4195691,8,7ee3a_00000


2024-08-31 16:53:29,106	WARNING tensorboardx.py:265 -- You are trying to log an invalid value (ray/tune/env_runners/episode_media/rendering=[<wandb.sdk.data_types.video.Video object at 0x3ef3c0090>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0550>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0590>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0650>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0690>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0750>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0790>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0890>, <wandb.sdk.data_types.video.Video object at 0x3ef3c08d0>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0a10>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0a50>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0d10>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0dd0>, <wandb.sdk.data_types.video.Video object at 0x3ef3c0f50>, <wandb.sdk.data_types.video.Video object at 0x3